In [1]:
!pip install langchain-community sentence-transformers faiss-cpu beautifulsoup4 requests

  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transform

In [7]:
!pip install -U langchain langchain-core langchain-community langchain-text-splitters


#### Import the necessary libraries. 

In [5]:
import re 
import requests 
from bs4 import BeautifulSoup 
from typing import List 

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS 

c:\rag_url_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import WebBaseLoader


c:\rag_url_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
MAIN_URL  = "https://www.rnc-pro.com/rnc-pro/pfm/100/192_0100.HTM"

main_loader = WebBaseLoader(MAIN_URL)
main_docs = main_loader.load()
main_docs

[Document(metadata={'source': 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0100.HTM', 'title': 'ISO Businessowners Program Overview', 'language': 'No language found.'}, page_content="\n\n\n\n\n\n\n\nISO Businessowners Program Overview\n\n\n\n\n\n\n\n\n\nISO BUSINESSOWNERS PROGRAM OVERVIEW\n(September 2022)\nINTRODUCTION \nThe Insurance Services\r\nOffice (ISO) Businessowners Program policy provides a package of insurance\r\ncoverages to smaller businesses similar to those that the ISO Commercial\r\nPackage Policy (CPP) offers. This program initially used separate property and\r\nliability coverage forms. It was changed into a unified policy approach in July\r\n2002.\n\xa0\nThe 07 13 edition\r\naddresses some of the concerns that inevitably arise when a variety of\r\ncoverages are combined in one policy. It reflects events and amendments\r\nimplemented in other coverage forms. Many changes in wording follow the\r\nstandardized language in other ISO commercial lines coverage forms. This\

In [6]:
main_docs[0].page_content[:1000]

'\n\n\n\n\n\n\n\nISO Businessowners Program Overview\n\n\n\n\n\n\n\n\n\nISO BUSINESSOWNERS PROGRAM OVERVIEW\n(September 2022)\nINTRODUCTION \nThe Insurance Services\r\nOffice (ISO) Businessowners Program policy provides a package of insurance\r\ncoverages to smaller businesses similar to those that the ISO Commercial\r\nPackage Policy (CPP) offers. This program initially used separate property and\r\nliability coverage forms. It was changed into a unified policy approach in July\r\n2002.\n\xa0\nThe 07 13 edition\r\naddresses some of the concerns that inevitably arise when a variety of\r\ncoverages are combined in one policy. It reflects events and amendments\r\nimplemented in other coverage forms. Many changes in wording follow the\r\nstandardized language in other ISO commercial lines coverage forms. This\r\napproach is logical. For example, a term used in the Businessowners Coverage\r\nForm should have the same definition and meaning as in a commercial property\r\ncoverage form. \nEL

In [13]:
import re 
from typing import List
from langchain_core.documents import Document

In [17]:
def extract_hyperlinks_from_documents(docs:List[Document])-> List[str]:
    """
    Extract all hyperlinks from Langschain Document objects. 
    """
    url_pattern = r'https?://[^\s")\]>]+'
    all_urls = []

    for doc in docs : 
        found_urls = re.findall(url_pattern , doc.page_content)
        all_urls.extend(found_urls)
    
    ## remove duplicates 

    return list(set(all_urls))


In [18]:
child_urls = extract_hyperlinks_from_documents(main_docs)
len(child_urls)

0

In [19]:
child_urls[:10]


[]

##### since , it is not possible to extract urls from that document using RAG or langchain . so , we would be using BeautifulSoup . 


- We switched to BeautifulSoup because:
- LangChain WebBaseLoader gave you clean text only
- Clean text removes hyperlinks
- To extract URLs, we must first work with raw HTML
- This is correct engineering reasoning, not a mistake.

In [23]:
import requests 
from bs4 import BeautifulSoup

MAIN_URL = "https://www.rnc-pro.com/rnc-pro/pfm/100/192_0100.HTM"

response = requests.get(MAIN_URL , timeout = 10)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

soup 

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Word.Document" name="ProgId"/>
<meta content="Microsoft Word 15" name="Generator"/>
<meta content="Microsoft Word 15" name="Originator"/>
<link href="192_0100_files/filelist.xml" rel="File-List"/>
<link href="192_0100_files/preview.wmf" rel="Preview"/>
<title>ISO Businessowners Program Overview</title>
<!--[if gte mso 9]><xml>
 <o:DocumentProperties>
  <o:Subject>192.1</o:Subject>
  <o:Author>Linda Ferguson/J. Hunter-Lattak, ed.</o:Author>
  <o:Keywords>192.1</o:Keywords>
  <o:Description>The Rough Notes Company, Inc.</o:Description>
  <o:Template>PF&amp;MTEXT</o:Template>
  <o:LastAuthor>Bruce Hicks</o:LastAuthor>
  <o:Revision>2</o:Revision>
  <o:TotalTim

In [25]:
### Extract all the hyperlinks from the page . 

links = []

for a_tag in soup.find_all("a" , herf = True):
    links.append(a_tag["herf"])

## remove duplicates . 
links = list(set(links))
links 

[]

#### since, herf links are not present on that html content which i have extracted.

In [27]:
## looking for the "a" tags in html content , since "a" tags contain links . 
len(soup.find_all("a"))


7

In [28]:
links = []

for a in soup.find_all("a"):
    href = a.get("href")
    if href and href.strip():
        links.append(href.strip())
    
links = list(set(links))
links

['..\\100\\192_0401.HTM',
 '..\\100\\192_0402.HTM',
 '..\\100\\192_0403.HTM',
 '..\\100\\192_0502.HTM',
 '..\\100\\192_0200.HTM',
 '..\\100\\192_0501.HTM']

### Lets first store the main Url content into data.txt file . 


In [29]:
## extract clean text from the main URL . 
main_text = soup.get_text(separator=" ")
main_text = " ".join(main_text.split()) ## remove extra spaces / newlines. 

## Write to data.txt  
with open("data.txt" , "w" , encoding = "utf-8") as f : 
    f.write(main_text)

In [30]:
len(main_text)


5732

In [31]:
with open("data.txt" , "r" , encoding = "utf-8") as f : 
    print(f.read() [:500])

ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers. This program initially used separate property and liability coverage forms. It was changed into a unified policy approach in July 2002. The 07 13 edition addresses some of the concerns that inevitab


I completed the ingestion phase by extracting clean textual content from the main URL and persisting it into a flat file (data.txt) to act as the initial knowledge source for the RAG pipeline.

In [33]:
### Extracting urls from main URL . 
response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, "html.parser")
soup

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Word.Document" name="ProgId"/>
<meta content="Microsoft Word 15" name="Generator"/>
<meta content="Microsoft Word 15" name="Originator"/>
<link href="192_0100_files/filelist.xml" rel="File-List"/>
<link href="192_0100_files/preview.wmf" rel="Preview"/>
<title>ISO Businessowners Program Overview</title>
<!--[if gte mso 9]><xml>
 <o:DocumentProperties>
  <o:Subject>192.1</o:Subject>
  <o:Author>Linda Ferguson/J. Hunter-Lattak, ed.</o:Author>
  <o:Keywords>192.1</o:Keywords>
  <o:Description>The Rough Notes Company, Inc.</o:Description>
  <o:Template>PF&amp;MTEXT</o:Template>
  <o:LastAuthor>Bruce Hicks</o:LastAuthor>
  <o:Revision>2</o:Revision>
  <o:TotalTim

In [34]:
## Extracted urls from the main url .
links = []

for a in soup.find_all("a"):
    href = a.get("href")
    if href and href.strip():
        links.append(href.strip())
    
links = list(set(links))
links

['..\\100\\192_0401.HTM',
 '..\\100\\192_0402.HTM',
 '..\\100\\192_0403.HTM',
 '..\\100\\192_0502.HTM',
 '..\\100\\192_0200.HTM',
 '..\\100\\192_0501.HTM']

#### now Let's extract content from this child urls . 


In [37]:
from urllib.parse import urljoin

def normalize_links(base_url, links):
    cleaned = []
    for link in links:
        # fix backslashes -> forward slashes
        link = link.replace("\\", "/")
        # resolve .. properly
        absolute_url = urljoin(base_url, link)
        cleaned.append(absolute_url)
    return list(set(cleaned))


In [38]:
child_urls = normalize_links(MAIN_URL, links)
child_urls


['https://www.rnc-pro.com/rnc-pro/pfm/100/192_0403.HTM',
 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0401.HTM',
 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0402.HTM',
 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0501.HTM',
 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0200.HTM',
 'https://www.rnc-pro.com/rnc-pro/pfm/100/192_0502.HTM']

In [41]:
import json
with open("data.txt", "a", encoding="utf-8") as f:
    for url in child_urls:
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            soup_child = BeautifulSoup(response.text, "html.parser")
            text = soup_child.get_text(separator=" ")
            text = " ".join(text.split())

            f.write(f"\n\n--- Source: {url} ---\n")
            f.write(text)

            print(f"Saved: {url}")

        except Exception as e:
            print(f"Failed to fetch {url}: {e}")


Saved: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0403.HTM
Saved: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0401.HTM
Saved: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0402.HTM
Saved: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0501.HTM
Failed to fetch https://www.rnc-pro.com/rnc-pro/pfm/100/192_0200.HTM: HTTPSConnectionPool(host='www.rnc-pro.com', port=443): Read timed out. (read timeout=10)
Saved: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0502.HTM


- Take main URL text

- Detect child URLs appearing inside main text

- Fetch child URL content

- Replace the child URL text inline with its actual content inside parentheses

- Save the final merged text to data.txt

In [43]:
import time
import requests
from bs4 import BeautifulSoup

def fetch_clean_text(url: str, retries: int = 3) -> str:
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=30)  # longer timeout
            response.raise_for_status()

            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text(separator=" ")
            return " ".join(text.split())

        except requests.exceptions.ReadTimeout:
            print(f"Timeout on {url}, retrying ({attempt + 1}/{retries})...")
            time.sleep(2)

    # final fallback if all retries fail
    return "[CONTENT NOT AVAILABLE DUE TO TIMEOUT]"


# STEP 1: start with main URL text
final_text = main_text

# STEP 2: replace each child URL inline with its content
for child_url in child_urls:
    try:
        child_content = fetch_clean_text(child_url)

        # optional: keep child content shorter to avoid explosion
        child_content = child_content[:1500]

        replacement = f"( {child_content} )"

        # replace URL wherever it appears in main text
        final_text = final_text.replace(child_url, replacement)

        print(f"Replaced content from: {child_url}")

    except Exception as e:
        print(f"Failed to process {child_url}: {e}")

# STEP 3: save final merged text
with open("data.txt", "w", encoding="utf-8") as f:
    f.write(final_text)


Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0403.HTM
Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0401.HTM
Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0402.HTM
Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0501.HTM
Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0200.HTM
Replaced content from: https://www.rnc-pro.com/rnc-pro/pfm/100/192_0502.HTM



- Main URL text order is preserved
- Child URL is replaced exactly where it appears
-  Child content is injected inside parentheses
- Multiple child URLs are handled
- Final data.txt contains ONE continuous document

In [44]:
import json

# 1. Read data.txt
with open("data.txt", "r", encoding="utf-8") as f:
    text = f.read()

# 2. Split text based on full stops
sentences = [s.strip() for s in text.split(".") if s.strip()]

# 3. Convert to list of strings format
# (using json ensures proper quotes and commas)
formatted_sentences = json.dumps(sentences, ensure_ascii=False)

# 4. Save to a new file
with open("data_sentences.txt", "w", encoding="utf-8") as f:
    f.write(formatted_sentences)


In [45]:
with open("data_sentences.txt", "r", encoding="utf-8") as f:
    print(f.read()[:500])


["ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers", "This program initially used separate property and liability coverage forms", "It was changed into a unified policy approach in July 2002", "The 07 13 edition addresses some of the concerns that 


I normalized the corpus by splitting the text into sentence-level units using full stops as delimiters and serialized them into a list structure for easier downstream processing

In [47]:
len(sentences)


37

In [49]:
sentences_check[:5]


['ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers',
 'This program initially used separate property and liability coverage forms',
 'It was changed into a unified policy approach in July 2002',
 'The 07 13 edition addresses some of the concerns that inevitably arise when a variety of coverages are combined in one policy',
 'It reflects events and amendments implemented in other coverage forms']

Since ,  Lancgchain needs Documents for performing operations like chunking , vectorDB, embeddings etc. 

#### Now Wrapping Each sentence into a LangChain Document . 

In [50]:
from langchain_core.documents import Document
import json 

## Load sentence list from file 
with open("data_sentences.txt" , "r" , encoding = "utf-8") as f : 
    sentences = json.load(f)

# Convert each sentence into LangChain Document . 

documents = [Document(page_content = s) for s in sentences]
len(sentences)

37

In [51]:
type(documents[0])

langchain_core.documents.base.Document

In [52]:
documents[0].page_content

'ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers'

After normalizing the corpus into sentence-level units, I converted each sentence into a LangChain Document to make the data compatible with downstream RAG components like chunking, embeddings, and retrieval. 

## Chunking 

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500 , ## Characters per chunk . 
    chunk_overlap = 50 ## overlap for context continuity . 
) 
chunks = text_splitter.split_documents(documents)
len(chunks)

39

In [55]:
type[chunks[0]]

type[Document(metadata={}, page_content='ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers')]

In [56]:
chunks[0].page_content

'ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers'

### Embeddings 

Creating Embeddings using Hugging Face . 


In [58]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


C:\Users\AsutoshKumar\AppData\Local\Temp\ipykernel_9924\124733854.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [59]:
### Quick check . 
embedding_model.embed_query("What is an insurance policy?")[:5]

[-0.05694250017404556,
 0.13338297605514526,
 -0.020222527906298637,
 0.0017461401876062155,
 0.062403082847595215]

After chunking the documents, I generated vector embeddings using a HuggingFace sentence-transformer model to enable semantic similarity search in the RAG pipeline

### Store the chunk embeddings in a FAISS vector database . 

Index these embeddings in a vector store . so we can later retrieve relevant chunks for questions. 

### Create FAISS Vector Store

In [61]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents = chunks , 
    embedding = embedding_model
)

In [62]:
type(vectorstore)

langchain_community.vectorstores.faiss.FAISS

In [63]:
vectorstore.index.ntotal

39

### Creating  a Retriever and Querying It

In [66]:
# create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# test query
query = "What is the ISO Businessowners Program?"

retrieved_docs = retriever.invoke(query)

# inspect results
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n--- Result {i} ---")
    print(doc.page_content[:400])



--- Result 1 ---
Related article: ISO Businessowners Program Rating Considerations

--- Result 2 ---
ISO Businessowners Program Overview ISO BUSINESSOWNERS PROGRAM OVERVIEW (September 2022) INTRODUCTION The Insurance Services Office (ISO) Businessowners Program policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers

--- Result 3 ---
Related Article: ISO Businessowners Program Available Endorsements and Their Uses UNDERWRITING Underwriting the ISO Businessowners Coverage Form begins by determining a given risk's eligibility


### Adding  an LLM and generating answers. 

In [67]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline 
from langchain_core.prompts import PromptTemplate

In [71]:
from transformers import pipeline

hf_pipeline = pipeline(
    "text2text-generation",   
    model="google/flan-t5-base",
    max_new_tokens=200
)


Device set to use cpu


In [73]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant that answers questions strictly based on the provided context.

Context:
{context}

Question:
{question}

Answer:
"""
)


In [74]:
# Ask a question
question = "What is the ISO Businessowners Program?"

# Retrieve relevant chunks
retrieved_docs = retriever.invoke(question)

# Combine retrieved text as context
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

# Generate answer using LLM
response = llm.invoke(
    prompt.format(context=context, question=question)
)

response


'policy provides a package of insurance coverages to smaller businesses similar to those that the ISO Commercial Package Policy (CPP) offers'

I implemented a complete RAG pipeline where documents were embedded into a FAISS vector store, relevant chunks were retrieved for a query, and a Hugging Face sequence-to-sequence model generated grounded answers based strictly on retrieved context